# Deep Learning from Scratch
# Ch.03 Neural Network

In [ ]:
import numpy as np
import sys, os
sys.path.append(os.pardir)    # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist
from PIL import Image    # Python image library (PIL) : 이미지 표시를 위한 모듈
import pickle
from common.functions import sigmoid, softmax

# 3.6 MNIST

# 3.6.1 MNIST Data Set

In [ ]:
# load_minst(.)가 MNIST 데이터를 받아와야 하므로 최초 실행 시에는 인터넷이 연결된 상태여야 한다. 
# 두 번째부터는 로컬에 저장된 pickle 파일을 읽어오므로 순식간에 끝난다.
'''
Pickle : 프로그램 실행 중에 특정 객체를 파일로 저장하는 파이썬의 기능.
    저장해둔 pickle 파일을 로드하면 실행 당시의 객체를 즉시 복원할 수 있다.
'''
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)    # (60000, 784)
print(t_train.shape)    # (60000,)
print(x_test.shape)    # (10000, 784)
print(t_test.shape)    # (10000,)

In [ ]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))    # 변환 : numpy 배열 → PIL용 데이터 객체
    pil_img.show()

img = x_train[0]
label = t_train[0]
print(label)    # 5

print("img.shape : ", img.shape)    # (784,)
img = img.reshape(28, 28)    # 형상을 원래 이미지의 크기로 변형
print("img.shape (reshaped) : ", img.shape)    # (28, 28)

img_show(img)

# 3.6.2 신경망의 추론 처리

In [ ]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)

    return x_test, t_test

# "sample_weight.pkl"에 저장된 학습된 가중치 매개변수를 읽어들인다.
# 이 파일에는 weight와 bias 매개변수가 딕셔너리 변수로 저장되어 있다.
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y)    # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

# 3.6.3 배치 처리

In [ ]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    
    return network

def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y

x, t = get_data()
network = init_network()

batch_size = 100    # Batch size
accuracy_cnt = 0

# Batch size만큼 묶어서 batch 지정 : x[0:100], x[100:200], ...
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    if i == 0:
        print("y_batch.shape : ", y_batch.shape)
    p = np.argmax(y_batch, axis=1)    # shape이 바뀌었기 때문에 axis=1 인수를 추가함.
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy : " + str(float(accuracy_cnt) / len(x)))

In [ ]:
# Shape and Axis
x = np.array([[0.1, 0.8, 0.1],
              [0.3, 0.1, 0.6],
              [0.2, 0.5, 0.3],
              [0.8, 0.1, 0.1]])
y_axis1 = np.argmax(x, axis=0)
y_axis2 = np.argmax(x, axis=1)
print("y_axis1 : ", y_axis1)    # 0.8[3], 0.8[0], 0.6[1]]
print("y_axis2 : ", y_axis2)    # 0.8[1], 0.6[2], 0.5[1], 0.8[0]